# 行列方程式のクロネッカー積による解法

運動制御モデルの論文を読んでいた際に，ある行列方程式の解法を忘却していたのでメモとして記事を書きました．

## クロネッカー積
### 定義と例
以下，Wikipediaの説明から引用する．

$A = (a_{ij})$ を $m \times n$ 行列，$B = (b_{kl})$ を $p \times q$ 行列とすると、それらのクロネッカー積 $A \otimes B$ は

$$
A\otimes B={\begin{bmatrix}a_{11}B&\cdots &a_{1n}B\\\vdots &\ddots &\vdots \\a_{m1}B&\cdots &a_{mn}B\end{bmatrix}}
$$

で与えられる $mp \times nq$ 区分行列である．例えば，

$$
\begin{bmatrix}1&2\\3&4\end{bmatrix} \otimes {\begin{bmatrix}0&5\\6&7\end{bmatrix}}={\begin{bmatrix}1\cdot 0&1\cdot 5&2\cdot 0&2\cdot 5\\1\cdot 6&1\cdot 7&2\cdot 6&2\cdot 7\\3\cdot 0&3\cdot 5&4\cdot 0&4\cdot 5\\3\cdot 6&3\cdot 7&4\cdot 6&4\cdot 7\\\end{bmatrix}}={\begin{bmatrix}0&5&0&10\\6&7&12&14\\0&15&0&20\\18&21&24&28
\end{bmatrix}}
$$

のような計算が成り立つ．

### 実装
Pythonでは[`numpy.kron()`](https://numpy.org/doc/stable/reference/generated/numpy.kron.html)，Juliaでは[Kronecker.jl](https://github.com/MichielStock/Kronecker.jl)を用いる．以下ではPythonを使用する．

In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True) # print時の設定

In [2]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[0, 5], [6, 7]])
print(np.kron(A, B))

[[ 0  5  0 10]
 [ 6  7 12 14]
 [ 0 15  0 20]
 [18 21 24 28]]


## 行列方程式の解法

### 1．$AX=B$
$AX=B$の時は簡単で，Aの逆行列を求めることができれば$X=A^{-1}B$となる．

In [3]:
X = np.linalg.inv(A) @ B
print(A @ X)

[[0. 5.]
 [6. 7.]]


### 2.  $AXB = C$
この記事の本題，クロネッカー積とRoth's column lemma (vec-trick)を用いる．Roth's column lemmaは，例えば，$A, B, C$が与えられていて，$X$を未知とするときの方程式 $AXB = C$を考えると，この方程式は

$$
(B^\top \otimes A)\text{vec}(X) = \text{vec}(AXB)=\text{vec}(C)
$$

の形に書き下すことができる，というものである．$\text{vec}(\cdot)$はvec作用素（行列を列ベクトル化する作用素）である．numpyだと`flatten()`で実現できる．Roth's column lemmaを用いれば，$AXB = C$の解は

$$
X = \text{vec}^{-1}\left((B^\top \otimes A)^{-1}\text{vec}(C)\right)
$$

として得られる．ただし，$\text{vec}(\cdot)^{-1}$は列ベクトルを行列に戻す作用素(inverse of the vectorization operator)である．numpyだと`reshape()`で実現できる (numpyのベクトルはデフォルトで行ベクトルであることに注意)．2つの作用素をまとめると，

$$
\begin{align}
\text{vec} &: R^{m\times n}\to R^{mn}\\
\text{vec}^{−1} &: R^{mn}\to R^{m×n}
\end{align}
$$

であり，$\text{vec}^{−1}\left(\text{vec}(X)\right)=X\ (\text{for all}\ X\in R^{m\times n})，\text{vec}\left(\text{vec}^{−1}(x)\right)=x\ (\text{for all}\ x \in R^{mn})$となる．

以下では$P, Q, R$が与えられている際に，方程式$PYQ=R$の解$Y$を求める．

In [4]:
m = 4
P = np.random.randn(m, m)
Q = np.random.randn(m, m)
R = np.arange(m**2).reshape(m, m)
print(R)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


上記に基づいて方程式を解く．

In [5]:
Y = (np.linalg.inv(np.kron(P, Q.T)) @ R.flatten()).reshape(m, m)
print(P @ Y @ Q)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


少々元の式と実装が異なるが，計算はできているので良しとしておく．

## 参考文献
- [クロネッカー積 - Wikipedia](https://ja.wikipedia.org/wiki/%E3%82%AF%E3%83%AD%E3%83%8D%E3%83%83%E3%82%AB%E3%83%BC%E7%A9%8D)
- [クロネッカー積とvec作用素とRoth's column lemma - でかいチーズをベーグルする](http://yamaguchiyuto.hatenablog.com/entry/2016/11/16/100000)